In [1]:
import os
import cv2
import time
import pandas as pd
from glob import glob
from concurrent.futures import ProcessPoolExecutor
#from vfill import write_raster, opencv,load_data_obj, split_data_and_mask,fill_rasterio

In [6]:
name = 'GEDI_L3'
fpath = "/media/ljp238/12TBWolf/OPENTOPOGRAPHY/OT_GLOBAL_DEMS_TILES/*/GEDI_L3/*.tif"
files = glob(fpath)

tile_dpath = "/home/ljp238/Downloads/GEDI_L3_BULK/tiles/"

In [7]:
dtm_global_fn = "/home/ljp238/Downloads/GEDI_L3_BULK/data/GEDI_L3_be/GEDI03_elev_lowestmode_mean_2019108_2022019_002_03_EPSG4326.tif"
chm_global_fn = "/home/ljp238/Downloads/GEDI_L3_BULK/data/GEDI_L3_vh/GEDI03_rh100_mean_2019108_2022019_002_03_EPSG4326.tif"

In [8]:
from rutils import get_raster_info, gdal_regrid
import numpy as np 
import math 
import rasterio 

In [11]:
import rasterio
import numpy as np
from rasterio.fill import fillnodata
import math

def load_raster(fpath):
    """
    Loads a raster dataset and reads the first band as a masked array.

    Parameters:
    fpath (str): Path to the raster file.

    Returns:
    numpy.ndarray: A NumPy array with nodata values set to np.nan.
    """
    with rasterio.open(fpath) as src:
        data = src.read(1, masked=True)
        data = data.filled(np.nan)  # Ensure nodata values are np.nan
    return data

def write_raster(output_path, data, reference_path):
    """
    Writes a NumPy array to a raster file, preserving metadata from a reference raster.

    Parameters:
    output_path (str): Path to save the output raster.
    data (numpy.ndarray): Data to write to the raster.
    reference_path (str): Path to the reference raster for metadata.
    """
    with rasterio.open(reference_path) as src:
        meta = src.meta.copy()

    # Update metadata
    meta.update(dtype=np.float32, nodata=np.nan)

    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(data, 1)

def fill_nodata(data, smoothing_iterations=0):
    """
    Fills nodata values in a raster using interpolation.

    Parameters:
    data (numpy.ndarray): Input raster data with nodata values.
    smoothing_iterations (int): Number of smoothing iterations. Defaults to 0.

    Returns:
    numpy.ndarray: Raster data with nodata values filled.
    """
    mask = np.isfinite(data)  # Valid data mask
    max_search_distance = int(math.sqrt(data.shape[0] ** 2 + data.shape[1] ** 2)) + 1

    return fillnodata(data, mask=mask, max_search_distance=max_search_distance, smoothing_iterations=smoothing_iterations)

def rio_fillna(input_path, output_path, smoothing_iterations=0):
    """
    Loads a raster, fills nodata values, and saves the result.

    Parameters:
    input_path (str): Path to the input raster.
    output_path (str): Path to save the output raster.
    smoothing_iterations (int): Number of smoothing iterations for nodata filling. Defaults to 0.
    """
    output_path = output_path.replace('.tif', f'_si{smoothing_iterations}.tif')
    
    # Load data
    data = load_raster(input_path)

    # Fill nodata
    filled_data = fill_nodata(data, smoothing_iterations)

    # Write output raster
    write_raster(output_path, filled_data, input_path)

# Example Usage:
# rio_fillna("input.tif", "output_filled.tif", smoothing_iterations=2)


In [14]:
for fi in files:
    tilename = fi.split('/')[-3]
    dout_path = os.path.join(tile_dpath,tilename)
    os.makedirs(dout_path, exist_ok=True)
    chm_tile = os.path.join(dout_path, f'{tilename}_{os.path.basename(chm_global_fn)}')
    dtm_tile = os.path.join(dout_path, f'{tilename}_{os.path.basename(dtm_global_fn)}')
    proj, xres, yres, xmin, xmax, ymin, ymax, w, h = get_raster_info(fi)
    #gdal_regrid(fi, chm_tile, xmin, ymin, xmax, ymax, xres, yres,mode="num", t_epsg='EPSG:4979', overwrite=False)
    #gdal_regrid(fi, dtm_tile, xmin, ymin, xmax, ymax, xres, yres,mode="num", t_epsg='EPSG:4979', overwrite=False)

    dtm_tile_fil = dtm_tile.replace('.tif', '_riofill.tif')
    chm_tile_fil = chm_tile.replace('.tif', '_riofill.tif')
    try:
        os.remove(dtm_tile_fil)
        os.remove(chm_tile_fil)
    except:
        continue
    rio_fillna(input_path=chm_tile, output_path=chm_tile_fil, smoothing_iterations=0)
    rio_fillna(input_path=dtm_tile, output_path=dtm_tile_fil, smoothing_iterations=0)

# functions from libb::to regrid this dataset by basefiles 
# riofill them and other unspervised filling  